## Imports

In [1]:
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *
from senmodel.metrics.train_metrics import *
from senmodel.train.train import *

In [2]:
SEED = 8642

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Data

In [3]:
BATCH_SIZE = 64

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

train_dataset = datasets.FashionMNIST(root='./data', train=True,
                                      download=True, transform=transform)
val_dataset = datasets.FashionMNIST(root='./data', train=False,
                                    download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

## Model

In [4]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=28 * 28, hidden_size=16, output_size=10):
        super(SimpleFCN, self).__init__()
        self.fc0 = nn.Linear(input_size, output_size)
        # self.fc1 = nn.Linear(hidden_size, output_size)
        # self.act = nn.ReLU()

    def forward(self, x):
        x = self.fc0(x)
        return x

In [5]:
model = SimpleFCN()
sparse_model = convert_dense_to_sparse_network(model, layers=[model.fc0])

## Train

In [6]:
hyperparams = {
    "choose_thresholds": {"fc0": 0.5},
    "num_epochs": 64,
    "metric": MagnitudeL2Metric(nn.CrossEntropyLoss()),
    "aggregation_mode": "mean",
    "replace_layers": ["fc0"],
    "threshold": 0.05,
    "min_delta_epoch_replace": 8,
    "window_size": 5,
    "lr": 1e-4,
    "delete_after": 5,    
}

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)

name

"choose_thresholds: {'fc0': 0.5}, num_epochs: 64, metric: MagnitudeL2Metric, aggregation_mode: mean, replace_layers: ['fc0'], threshold: 0.05, min_delta_epoch_replace: 8, window_size: 5, lr: 0.0001, delete_after: 5"

In [7]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: down-shift to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [8]:
run = wandb.init(
    project="self-expanding-nets",
    name=f"{name}",
)

In [9]:
train_sparse_recursive(sparse_model, train_loader, train_loader, val_loader, hyperparams)

100%|██████████| 938/938 [00:06<00:00, 155.38it/s]


Epoch 1/64, Train Loss: 1.1729, Val Loss: 0.8424, Val Accuracy: 0.7286


100%|██████████| 938/938 [00:05<00:00, 161.75it/s]


Epoch 2/64, Train Loss: 0.7434, Val Loss: 0.6969, Val Accuracy: 0.7683


100%|██████████| 938/938 [00:05<00:00, 168.77it/s]


...

Epoch 62/64, Train Loss: 0.3826, Val Loss: 0.4302, Val Accuracy: 0.8501


100%|██████████| 938/938 [00:13<00:00, 70.78it/s]


Epoch 63/64, Train Loss: 0.3816, Val Loss: 0.4296, Val Accuracy: 0.8511


100%|██████████| 938/938 [00:13<00:00, 67.39it/s]


Epoch 64/64, Train Loss: 0.3813, Val Loss: 0.4299, Val Accuracy: 0.8501


In [10]:
wandb.finish()

acc amount,▇▇███▆▆▆▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
del_len_choose,▄█▅▂▁▁▁▁▁
len_choose,▄█▅▂▁▁▁▁▁▂
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
params amount,▁▁▁▁▁▂▂▂▂▅▅▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████
params ratio,▁▃▃▄▅▄▄▄▅▆▇▇███████████████████████████▇
params to replace amount,█▆▅▄▃▃▄▅▆▆▆▅▄▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂
train loss,█▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train time,▁▁▁▁▁▂▂▃▂▄▄▃▃▄▅▄▅▅▄▅▅▄▅▇▆▅▅▅▅▅▅▆▆▆▆▇██▇▇
val accuracy,▁▃▄▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████████
val loss,█▆▅▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
